# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key

%matplotlib inline

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read the data
weather_df = pd.read_csv("../WeatherPy/weather.csv",index_col=0)

weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,63.19,71,0,7.36,AU,1616888878
1,coquimbo,-29.9533,-71.3436,59.00,88,75,9.22,CL,1616888879
2,pandan,14.0453,124.1694,80.26,83,84,8.03,PH,1616888880
3,husavik,66.0449,-17.3389,19.40,86,90,13.80,IS,1616888881
4,albany,42.6001,-73.9662,54.00,56,79,1.90,US,1616888882


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Gmaps
gmaps.configure(api_key = g_key)
# Heatmap location
location = weather_df[["Lat","Lng"]]
humidity = weather_df["Humidity"].astype(float)
print(humidity.max())

100.0


In [4]:
#Humidity heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location,
                                 weights=humidity,
                                dissipating = False,
                                max_intensity = 100,
                                point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# * Narrow down the DataFrame to find ideal weather condition for my vacation.
#   * A max temperature lower than 100 degrees but higher than 80.
#   * Wind speed up to 15 mph.
#   * Humidity from 40% to 65%
weather_df.loc[weather_df["Max Temp"]>80]
weather_df["my_temp"] = weather_df["Max Temp"].between(80, 100, inclusive = False)
weather_df["my_wind"] = weather_df["Wind Speed"].between(0, 15, inclusive = True)
weather_df["my_humidity"] = weather_df["Humidity"].between(40, 65, inclusive = True)

weather_df
my_places =[]
for row in range(len(weather_df)):
    if weather_df.iloc[row,9] == True:
        if weather_df.iloc[row,10] == True:
            if weather_df.iloc[row,11] == True:
                my_places.append(weather_df.iloc[row,:])
                
my_place_df = pd.DataFrame(my_places)

#Drop any rows will null values.
my_place_df.dropna()    
my_place_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,my_temp,my_wind,my_humidity
10,barra,-11.0894,-43.1417,83.07,43,60,2.28,BR,1616888887,True,True,True
146,emerald,-23.5333,148.1667,82.00,65,8,6.91,AU,1616888976,True,True,True
166,coahuayana,18.7333,-103.6833,80.60,44,40,11.50,MX,1616888989,True,True,True
214,tautira,-17.7333,-149.1500,89.60,62,40,10.36,PF,1616889022,True,True,True
241,kyabe,9.4515,18.9449,80.89,55,48,3.11,TD,1616889041,True,True,True
257,tijara,27.9333,76.8500,80.01,50,0,1.45,IN,1616889053,True,True,True
307,ewa beach,21.3156,-158.0072,84.20,48,40,10.36,US,1616889086,True,True,True
390,kokopo,-4.3520,152.2633,84.29,63,100,4.52,PG,1616889138,True,True,True
398,nanakuli,21.3906,-158.1547,84.20,48,40,10.36,US,1616889142,True,True,True
399,alamor,-4.0181,-80.0197,84.20,61,75,6.91,EC,1616889143,True,True,True


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = my_place_df[["City","Lat","Lng","Country"]]

In [9]:
hotel_df["Hotel Name"] = "" 
hotel_df

<ipython-input-9-035905bd69f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Country,Hotel Name
14,kaduna,10.5222,7.4383,NG,
47,santiago del estero,-27.7951,-64.2615,AR,
82,kavaratti,10.5669,72.6420,IN,
84,guadalupe,25.6833,-100.2500,MX,
105,hithadhoo,-0.6000,73.0833,MV,
157,kieta,-6.2167,155.6333,PG,
197,voh,-20.9667,164.7000,NC,
203,paita,-5.0892,-81.1144,PE,
245,poya,-21.3500,165.1500,NC,
305,faanui,-16.4833,-151.7500,PF,


In [7]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "not found"
hotel_df

C:\Users\meria\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\meria\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Lat,Lng,Country,Hotel Name
10,barra,-11.0894,-43.1417,BR,Motel Albatroz
146,emerald,-23.5333,148.1667,AU,The Irish Village
166,coahuayana,18.7333,-103.6833,MX,El rranchito michuacan de ocampos
214,tautira,-17.7333,-149.1500,PF,Pension Oaoa
241,kyabe,9.4515,18.9449,TD,not found
257,tijara,27.9333,76.8500,IN,"Tijara Fort-Palace - 19th Century, Alwar"
307,ewa beach,21.3156,-158.0072,US,Uncle Junior Robert
390,kokopo,-4.3520,152.2633,PG,Kokopo Beach Bungalow Resort
398,nanakuli,21.3906,-158.1547,US,Camp Pālehua
399,alamor,-4.0181,-80.0197,EC,Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)

# Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))